# T&C Source code walkthrough

This notebook goes through the T&C source code, in the order in which it would be executed during a typical simulation, and attempts to explain what different sections of the code do.
It furthermore tries to connect parts of the code to corresponding sections in the T&C technical reference.

The code is based on [T&C Version 4.0](https://codeocean.com/capsule/0294088/tree/v4), released on Jan 28, 2025. Line numbers refer to the examples provided and might be slightly different in other versions of the code. 
Comments added to the code blocks as part of this notebook start with `%!` to distinguish them from comments originally in the code.
(Hyper-)parameters that are meant to be modified are marked with a `<-` in the comment.

To inspect variables interactively you can always add a new code cell inside this notebook.

## Files that need to be created or modified before running the code

### Biogeochemistry_IO.m

On line 48, `load(...)`, needs to contain the path to a file containing deposition data for nutrients. It is usually named `All_deposition_data.mat`.

### Meteorological inputs
A file containing meteorological variables for the given site, such as precipitation and temperatures. Often named something with "data" and "run". This needs to be generated as part of the pre-processing. Here we will use `Data_Run_Zurich_Fluntern.mat` from the `Inputs` directory.

### Deposition data
Often named `All_deposition_data.mat`. Contains deposition data for nutrients. It is site-independent and can be copied from other projects.

### CO2 concentration data
Usually `Ca_Data.mat`. Site-independent, can be copied from other projects.

### Parameter file
Usually named `MOD_PARAM_<site_identifier>.m`. Contains all relevant parameters for the simulation, such as soil properties, plant characteristics, and some initial conditions.
`TODO elsewere`: Describe parameters in that file.

### Prova-File

Usually named `prova_<site_identifier>.m`. This file contains a range of simulation hyper-parameters, but will also be used to start the simulation and execute the main function.
We will use `prova_Rural_Zurich`, which comes as an example with the T&C code, and can be found in the folder `PARAMETER_PLOT_FILES`. We will go through it here. Most parts are very straightforward so that explanations will be given directly as comments added to the code.

If using relative paths, the prova file needs to be executed from within its directory and the paths need to be relative to that.

In [31]:
cd ../PARAMETER_PLOT_FILES/

#### Lines 1-21

In [32]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%% WORKING LAUNCH PAD HBM  %%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%% INPUT MANAGER 
current_directory = cd;  %! Store current directory. Will be used later to navigate back to it.
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%
NN= 8760;  %! <- number of time steps (in hours)
%%%%%%%%% Time step 
dt=3600; %%[s] number of seconds per time step
dth=1; %%[h] number of hours per time step
%%%%%%%%%%%%
ms=16; %%% <- Number of soil Layers (must mach length of `Zs` - 1 in parameter file)
cc = 1; %! <- Number of (vegetation) crown areas as described in TR 2.2 - Vegetation composition
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%% METEO INPUT
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
id_location =  'ZURICH_SMA';  %! <- This should correspond to the last part of the file name of the parameter file, i.e. here it would be "MOD_PARAM_ZURICH_SMA.mat"
load('../data/Inputs/Data_Run_Zurich_Fluntern.mat')  %! <- This path needs to point to the file containing meteorological inputs
Date=D; clear D %! This line is only necessary if the date variable in the meteorological data is not already called "Date"
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

If your meteorological data contains longwave radiation measurements (`Latm`), you can add the following  line below this, so that the simulation does not need to calculate this value from cloud cover:

#### Lines 22-34: Selection of time range
Set the time range of the simulation by selecting the variables `x1` and `x2`. They correspond to the indices of the first and last time step (wrt to the dates in the `Date` variable) to be part of the simulation. Note that this is where the correspondence between time step index and calendar dates is defined. The remainder of this code block ensures that the correct subset of the meteorological variables is selected. Note that this should match the number of timesteps `NN`, i.e. `x2` - `x1` = `NN`. You could also ensure this by setting `NN` programmatically.

In [3]:
%%%%%%%%%%%%%%%%%%
x1=1;  %! <- Index of the first time step wrt. `Date`
x2= 8760;  %! <- Index of the last time step wrt. `Date`
%% Note that x2-x1 should match NN-1
%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%5
Date=Date(x1:x2);
Pr=Pr(x1:x2);
Ta=Ta(x1:x2);
Ws=Ws(x1:x2); ea=ea(x1:x2);  SAD1=SAD1(x1:x2);
SAD2=SAD2(x1:x2); SAB1=SAB1(x1:x2); Pre=Pre(x1:x2);
SAB2=SAB2(x1:x2); N=N(x1:x2); Tdew=Tdew(x1:x2);esat=esat(x1:x2);
PARB=PARB(x1:x2); PARD = PARD(x1:x2);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

If you want to define a date range explicitly (hourly precision), you can do so by replacing the definitions of `x1` and `x2` with the following code, which has been provided by Taiqi Li (in this example from 0:00 Jan 1st 2005 to 23:00 Dec 31st 2014):

#### Lines 35-36: Sun integration time
The variables `t_bef` and `t_aft` the start and end of the integration interval for solar variables. They are in hours, relative to the recorded timestep in `Date`. Note that `t_bef` moves backward in time (i.e. negative values are *later*), whereas `t_aft` is in positive time direction. These values are typically obtained from the meteorological dataset (e.g. ERA5).

In [4]:
%%%%%%%%%%%
t_bef= -0.67; t_aft= 1.67;  %! <- Solar integration time limits. Usually obtained with meteorological data

#### Lines 37-39: Calculate vapor pressure deficit
No need to change anything here

In [5]:
%%%%%%%%%%%%%%%%%%%%
Ds=esat-ea; %% [Pa] Vapor Pressure Deficit
Ds(Ds<0)=0;

#### Lines 40-45: CO2 data and oxygen partial pressure
In line 41, the path to the file containing CO2 data has to be provided. The following code loads the data and keeps only the values for the date range of the simulation.

In [7]:
%%%%%%%%%%%%%%%%%%%%%%%%%
load('../data/Inputs/Ca_Data.mat'); %! <- Insert path to CO2 data file here
d1 = find(abs(Date_CO2-Date(1))<1/36);d2 = find(abs(Date_CO2-Date(end))<1/36);
Ca=Ca(d1:d2); 
clear d1 d2 Date_CO2 
Oa= 210000;% Intercellular Partial Pressure Oxygen [umolO2/mol] -

#### Lines 46-51 Wind speed and explicit date
Nothing to do here.
Line 46 sets any wind speed value less or equal to zero to a small value. The following lines generate a vector with explicit date values (year, month, day, and hour).

In [8]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Ws(Ws<=0)=0.01;
%%dt,Pr(i),Ta(i),Ws(i),ea(i),Pre(i),Rdir(i),Rdif(i),N(i),z,Tdew(i),esat(i),.
[YE,MO,DA,HO,MI,SE] = datevec(Date);
Datam(:,1) = YE; Datam(:,2)= MO; Datam(:,3)= DA; Datam(:,4)= HO;
clear YE MO DA HO MI SE

#### Lines 52-65: Load parameters, run main program, and save results
Lines 53 and 56 define the path to the parameter file and the directory of the model source code. They might have to be updated.
The program will change to the source code directory and run `MAIN_FRAME` which executes the actual simulation with the loaded parameters.
It will then save all variables (inputs and outputs) and return to the original directory.
You can set the path to the output file in line 61. If using a relative path, keep in mind that at this point the program will be in the source code directory. (In some versions of the code this is different, and `cd(current_directory)` is executed *before* saving the results.

In [33]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
PARAM_IC = strcat(current_directory,'/MOD_PARAM_',id_location); %! <- must point to parameter file

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Directory='../T&C_CODE'; %! <- must point to source code directory
cd(Directory)

We will not execute the following code because we will execute the `MAIN_FRAME` code directly here in the cells of this notebook.

%%%%%%%%%%%%%%%%%%%%%%%%%%%%
MAIN_FRAME ;

%%%%%%%%%%%%%%%%%%%%%%%%%%%%
file_to_save = strcat('../results/Ris_',id_location,'.mat'); %! <- set output file here
save(file_to_save);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%
cd(current_directory);

## MAIN_FRAME

After the prova file has set up everything it will execute this file, which runs the actual simulation. 

#### Lines 1-176
`MAIN_FRAME` starts by initializing a range of variables. Most of them are arrays that will hold the values of timeseries. Their length is thus `NN`, the number of simulation time steps. For others, their length is `NNd` which is the number of days that are going to be simulated.

In [13]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%% MAIN_FRAME OF HBM-VEG %%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%% INITIZIALIZATION VARIABLES
%%%%%--->>  INIZIALIZATION VARIABLES  <<--- %%%%%%%%%%%%%%%%%%%%%%%
%%% j time dt = 1 day  %%%%%%%%%%%%%%%%
%%% i time dt = 1h
%%% ms soil layer
%%% cc Crown Area number
%%% NN time step
dtd = 1; %% [day]
dth = dt/3600; %% [hour] %! Number of hours per time step
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
V=zeros(NN,ms);
O=zeros(NN,ms);
Qi_out=zeros(NN,ms);
WTR=zeros(NN,ms);
POT=zeros(NN,ms);
Tdp=zeros(NN,ms);
%Sdp=zeros(NN,ms);
Vice=zeros(NN,ms);
Oice=zeros(NN,ms);
%%%%%%%%%%%%%%%%%%%%%%%%%%%
OF=zeros(NN,1); OS=zeros(NN,1);
ZWT=zeros(NN,1);
Pr_sno=zeros(NN,1); Pr_liq=zeros(NN,1);
er=zeros(NN,1);
Ts=zeros(NN,1);ra=zeros(NN,1);r_soil=zeros(NN,1);
b_soil=zeros(NN,1); Tice=zeros(NN,1);
Rn=zeros(NN,1);H=zeros(NN,1);
QE=zeros(NN,1);Qv=zeros(NN,1); Lpho=zeros(NN,1);
EG=zeros(NN,1); ESN=zeros(NN,1);ESN_In=zeros(NN,1);
EWAT=zeros(NN,1);EIn_urb=zeros(NN,1);
EIn_rock=zeros(NN,1); dw_SNO=zeros(NN,1);
G=zeros(NN,1); SWE=zeros(NN,1);
SND=zeros(NN,1);%snow_alb=zeros(NN,1);
ros=zeros(NN,1);In_SWE=zeros(NN,1);SP_wc=zeros(NN,1);
WR_SP=zeros(NN,1);U_SWE=zeros(NN,1);NIn_SWE=zeros(NN,1);
dQ=zeros(NN,1);Qfm=zeros(NN,1);t_sls=zeros(NN,1);
DQ=zeros(NN,1);DT=zeros(NN,1);
In_urb=zeros(NN,1);In_rock=zeros(NN,1);
SE_rock=zeros(NN,1);SE_urb=zeros(NN,1);
Csno=zeros(NN,1);WIS=zeros(NN,1);
Lk=zeros(NN,1);f=zeros(NN,1);
Rh=zeros(NN,1);Rd=zeros(NN,1);
NDVI=zeros(NN,1);alp_soil=zeros(NN,1);
tau_sno=zeros(NN,1);e_sno=zeros(NN,1);ALB=zeros(NN,1);
EK=zeros(NN,1);
dQVEG=zeros(NN,1);TsV=zeros(NN,1);
HV=zeros(NN,1);QEV=zeros(NN,1);
Cice=zeros(NN,1);
Lk_wat=zeros(NN,1); Lk_rock=zeros(NN,1);
EICE=zeros(NN,1);
WAT=zeros(NN,1);ICE=zeros(NN,1);ICE_D=zeros(NN,1);
WR_IP=zeros(NN,1);NIce=zeros(NN,1);Cicew=zeros(NN,1);
IP_wc=zeros(NN,1);
Csnow=zeros(NN,1);FROCK=zeros(NN,1);
Imelt=zeros(NN,1);Smelt=zeros(NN,1);
Tdamp=zeros(NN,1); Gfin=zeros(NN,1);
In_Litter=zeros(NN,1); ELitter=zeros(NN,1);
Ws_under=zeros(NN,1);
Ts_under=NaN*ones(NN,1);
Tdpsnow = zeros(NN,5);
if  not(exist('IrD','var'))
    IrD=zeros(NN,1);
end
if  not(exist('Salt','var'))
    Salt=zeros(NN,1); %%% Salt = Salt Concentration [mol Salt/ m-3] 
end
NetWatWet=zeros(NN,1);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
r_litter=zeros(NN,cc);
%%%
In_H=zeros(NN,cc);In_L=zeros(NN,cc);
OH=zeros(NN,cc);OL=zeros(NN,cc);
Psi_s_H=zeros(NN,cc); Psi_s_L=zeros(NN,cc);
Tdp_H=zeros(NN,cc);Tdp_L=zeros(NN,cc);
rap_H=zeros(NN,cc);rap_L=zeros(NN,cc);
Dr_H=zeros(NN,cc);Dr_L=zeros(NN,cc);
T_H=zeros(NN,cc);T_L=zeros(NN,cc);EIn_H=zeros(NN,cc);EIn_L=zeros(NN,cc);
rb_H=zeros(NN,cc);rb_L=zeros(NN,cc);
An_H=zeros(NN,cc);Rdark_H=zeros(NN,cc);
An_L=zeros(NN,cc);Rdark_L=zeros(NN,cc);
Ci_sunH=zeros(NN,cc);Ci_sunL=zeros(NN,cc);
Ci_shdH=zeros(NN,cc);Ci_shdL=zeros(NN,cc);
rs_sunH=zeros(NN,cc);rs_sunL=zeros(NN,cc);
rs_shdH=zeros(NN,cc);rs_shdL=zeros(NN,cc);
%%%%%
Vx_H=zeros(NN,cc);Vl_H=zeros(NN,cc);
Vx_L=zeros(NN,cc);Vl_L=zeros(NN,cc);
Psi_x_H=zeros(NN,cc);Psi_l_H=zeros(NN,cc);
Psi_x_L=zeros(NN,cc);Psi_l_L=zeros(NN,cc);
gsr_H=zeros(NN,cc);
Jsx_H=zeros(NN,cc); Jxl_H=zeros(NN,cc);
Kleaf_H=zeros(NN,cc);Kx_H=zeros(NN,cc);
gsr_L=zeros(NN,cc);
Jsx_L=zeros(NN,cc);Jxl_L=zeros(NN,cc);
Kleaf_L=zeros(NN,cc);Kx_L=zeros(NN,cc);
fapar_H=zeros(NN,cc);fapar_L=zeros(NN,cc);
SIF_H=zeros(NN,cc);SIF_L=zeros(NN,cc);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
NCP = 8; %% Number of Carbon Pool
NNd = ceil(NN/24)+1;
LAI_L=zeros(NNd,cc); B_L=zeros(NNd,cc,NCP); NPP_L=zeros(NNd,cc);Rg_L=zeros(NNd,cc);
RA_L=zeros(NNd,cc);Rmc_L=zeros(NNd,cc);ANPP_L=zeros(NNd,cc);
Rms_L=zeros(NNd,cc);Rmr_L=zeros(NNd,cc); PHE_S_L=zeros(NNd,cc); dflo_L=zeros(NNd,cc);
AgeL_L=zeros(NNd,cc);e_rel_L=ones(NNd,cc);SAI_L=zeros(NNd,cc); hc_L=zeros(NNd,cc); e_relN_L=ones(NNd,cc); BA_L=zeros(NNd,cc);
%%%
LAI_H=zeros(NNd,cc); B_H=zeros(NNd,cc,NCP);NPP_H=zeros(NNd,cc);Rg_H=zeros(NNd,cc);
RA_H=zeros(NNd,cc);Rms_H=zeros(NNd,cc);Rmr_H=zeros(NNd,cc); ANPP_H=zeros(NNd,cc);
PHE_S_H=zeros(NNd,cc); dflo_H=zeros(NNd,cc);Rmc_H=zeros(NNd,cc);
AgeL_H=zeros(NNd,cc);e_rel_H=ones(NNd,cc);SAI_H=zeros(NNd,cc); hc_H=zeros(NNd,cc); e_relN_H=ones(NNd,cc); BA_H=zeros(NNd,cc);
%%%
Rrootl_H=zeros(NNd,cc); Rrootl_L=zeros(NNd,cc);
Bfac_dayH=ones(NNd,cc); Bfac_weekH=ones(NNd,cc); NPPI_H=zeros(NNd,cc); TdpI_H=zeros(NNd,cc);
Bfac_dayL=ones(NNd,cc); Bfac_weekL=ones(NNd,cc); NPPI_L=zeros(NNd,cc); TdpI_L=zeros(NNd,cc);
%%%
Sr_H=zeros(NNd,cc);  Slf_H=zeros(NNd,cc);
Sfr_H=zeros(NNd,cc); Swm_H=zeros(NNd,cc);  Sll_H=zeros(NNd,cc);
Sr_L=zeros(NNd,cc); Slf_L=zeros(NNd,cc);
Sfr_L=zeros(NNd,cc); Swm_L=zeros(NNd,cc); Sll_L=zeros(NNd,cc);
LAIdead_H=zeros(NNd,cc); LAIdead_L=zeros(NNd,cc);
Rexmy_H=zeros(NNd,cc,3); AgeDL_H=zeros(NNd,cc); Nreserve_H=zeros(NNd,cc);
Preserve_H=zeros(NNd,cc); Kreserve_H=zeros(NNd,cc);
rNc_H=zeros(NNd,cc);rPc_H=zeros(NNd,cc);rKc_H=zeros(NNd,cc);
Rexmy_L=zeros(NNd,cc,3); AgeDL_L=zeros(NNd,cc); Nreserve_L=zeros(NNd,cc);
Preserve_L=zeros(NNd,cc); Kreserve_L=zeros(NNd,cc);
rNc_L=zeros(NNd,cc);rPc_L=zeros(NNd,cc);rKc_L=zeros(NNd,cc);
NBLeaf_H =zeros(NNd,cc); PARI_H=zeros(NNd,cc,3) ; NBLI_H=zeros(NNd,cc);
NBLeaf_L =zeros(NNd,cc);PARI_L=zeros(NNd,cc,3) ; NBLI_L=zeros(NNd,cc);
%%%%%
NupI_H=zeros(NNd,cc,3);
NupI_L=zeros(NNd,cc,3);
NavlI=zeros(NNd,3);
%%%
RB_L=zeros(NNd,cc,7);
RB_H=zeros(NNd,cc,7);
NuLit_H =zeros(NNd,cc,3);
NuLit_L =zeros(NNd,cc,3);
%%%%%
BLit=zeros(NNd,cc);
%%%%%
Nuptake_H=zeros(NNd,cc);
Puptake_H=zeros(NNd,cc);
Kuptake_H=zeros(NNd,cc);
FNC_H=zeros(NNd,cc);
TexC_H=zeros(NNd,cc);TexN_H=zeros(NNd,cc);TexP_H=zeros(NNd,cc);TexK_H=zeros(NNd,cc);
TNIT_H=zeros(NNd,cc);TPHO_H=zeros(NNd,cc);TPOT_H=zeros(NNd,cc);
SupN_H=zeros(NNd,cc);SupP_H=zeros(NNd,cc);SupK_H=zeros(NNd,cc);
%%%%
Nuptake_L=zeros(NNd,cc);
Puptake_L=zeros(NNd,cc);
Kuptake_L=zeros(NNd,cc);
FNC_L=zeros(NNd,cc);
TexC_L=zeros(NNd,cc);TexN_L=zeros(NNd,cc);TexP_L=zeros(NNd,cc);TexK_L=zeros(NNd,cc);
TNIT_L=zeros(NNd,cc);TPHO_L=zeros(NNd,cc);TPOT_L=zeros(NNd,cc);
SupN_L=zeros(NNd,cc);SupP_L=zeros(NNd,cc);SupK_L=zeros(NNd,cc);
%%%%
ISOIL_H=zeros(NNd,cc,18);
ISOIL_L=zeros(NNd,cc,18);
ManIH = zeros(cc,1);
ManIL = zeros(cc,1);

AgrHarNut =  zeros(NNd,3);
%%%%%%%%%%%%%%%%%
jDay=zeros(NNd,1);L_day=zeros(NNd,1);
%%%%
Ccrown_t =ones(NNd,cc);
AgePl_H =zeros(NNd,cc); AgePl_L =zeros(NNd,cc);
Tden_H =zeros(NNd,cc); Tden_L =zeros(NNd,cc);
TBio_Ht =zeros(NNd,cc); TBio_Lt =zeros(NNd,cc);
ZR95_Ht =zeros(NNd,cc); ZR95_Lt =zeros(NNd,cc);

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Lines 177-220: Optional soil biogeochemistry initialization
From these lines it seems like in order to run the biogeochemistry module, one needs to set `OPT_BG=1` somewhere in the prova file before executing `MAIN_FRAME`.
If that is the case, an internal flag `OPT_SoilBiogeochemistry` will be set to 1, and some additional arrays will be initialized. Otherwise `OPT_SoilBiogeochemistry` is set to 0.

In [15]:

if  not(exist('OPT_BG','var'))
    OPT_SoilBiogeochemistry = 0;
else
    if  OPT_BG == 0
        OPT_SoilBiogeochemistry = 0;
    else
        OPT_SoilBiogeochemistry = 1;
    end 
end
%%%%%%%%%%%%%
if OPT_SoilBiogeochemistry == 1
    P=zeros(NNd,55);
    R_litter=zeros(NNd,1);
    R_microbe=zeros(NNd,1);
    R_litter_sur=zeros(NNd,1);
    R_ew=zeros(NNd,1);
    VOL=zeros(NNd,1);
    N2flx = zeros(NNd,1);
    Min_N = zeros(NNd,1);
    Min_P = zeros(NNd,1);
    R_bacteria= zeros(NNd,1);
    RmycAM = zeros(NNd,1);
    RmycEM = zeros(NNd,1);
    Prod_B = zeros(NNd,1);
    Prod_F = zeros(NNd,1);
    BfixN = zeros(NNd,1);
    LitFirEmi =  zeros(NNd,2);
    LEAK_NH4 = zeros(NNd,1);
    LEAK_NO3 = zeros(NNd,1);
    LEAK_P = zeros(NNd,1);
    LEAK_K = zeros(NNd,1);
    LEAK_DOC = zeros(NNd,1);
    LEAK_DON = zeros(NNd,1);
    LEAK_DOP = zeros(NNd,1);
    R_NH4 = zeros(NNd,1);
    R_NO3 = zeros(NNd,1);
    R_P = zeros(NNd,1);
    R_K = zeros(NNd,1);
    R_DOC = zeros(NNd,1);
    R_DON = zeros(NNd,1);
    R_DOP = zeros(NNd,1);
    RexmyI=zeros(NNd,3);
end

#### Line 222-284: Parameter and optionssetup
First, loads parameters defined in the parameter file. Then executes another script from the `T&C_CODE` directory, named `Restating_parameters` to check for a range of parameters if they already exist and, if not, initialize them with default parameters.
It will also run `Check_Land_Cover_Fractions` to make sure that land cover fractions add up to 1, and initialize a few further variables, including `j`, which seems to be a running variable indicating the current day.
Finally, a few options that seem to determine the control flow of the program as well as optimization options and options for ODE solvers are set.

In [43]:
%%%%%%%%% CALL PARAMETERS AND INITIAL CONDITION
run(PARAM_IC);
Restating_parameters;
if length(Oice)==1
    Oice=zeros(NN,ms);
end
%%%%%%%%%%
Tdeb =zeros(NN,max(1,length(Zs_deb)-1));
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Check_Land_Cover_Fractions(Crock,Curb,Cwat,Cbare,Ccrown);
CcrownFIX = Ccrown;
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%% Lateral Contribution
q_runon=zeros(NN,1); %%[mm/h]
Qi_in=zeros(NN,ms); %%[mm/h]
%%%%%%%%%%%%%%%%%%%%%%%%%
tic ;
CK1=zeros(NN,1);  CK2=zeros(NN,1);
%profile on
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
j=1; %%% time dt = 1 day  %%%%%%%%%%%%%%%%
Tstm0= Ts(1);
%%% i time dt = 1h
%%% ms soil layer
%%% cc Crown Area present
%%%%%%%%%%%%%%%% NUMERICAL METHODS OPTIONS
%Opt_CR = optimset('TolX',3);
%Opt_ST = optimset('TolX',0.1);
%Opt_ST = optimset('TolX',0.1,'Display','iter');
Opt_CR = optimset('TolFun',1);%,'UseParallel','always');
Opt_ST = optimset('TolFun',0.1);%,'UseParallel','always');
Opt_ST2 = optimset('TolFun',0.1,'Display','off');
OPT_SM=  odeset('AbsTol',0.05,'MaxStep',dth);
OPT_VD=  odeset('AbsTol',0.05);
OPT_PH= odeset('AbsTol',0.01);
OPT_STh = odeset('AbsTol',0.02);
OPT_VegSnow = 1;
OPT_SoilTemp = 1;
OPT_FR_SOIL = 1; 
OPT_min_SPD = Inf; %% [m] minimum snow pack depth to have a multilayer snow 
%%%%
if  not(exist('OPT_VCA','var'))
    OPT_VCA = 0;
    OPT_ALLOME = 0;
end
if  not(exist('OPT_DROOT','var'))
    OPT_DROOT = 0;
end
if  not(exist('OPT_PlantHydr','var'))
    OPT_PlantHydr = 0;
end
if  not(exist('OPT_EnvLimitGrowth','var'))
    OPT_EnvLimitGrowth = 0;
end
if  not(exist('OPT_WET','var'))
    OPT_WET = 0;
else
    if  not(exist('Wlev','var'))
        Wlev=zeros(NN,1);
    end
    Wlevm1 = Rd(1)+Rh(1);
end
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Lines 285-556: Main loop
Interestingly, this starts at hour `2`. Presumably hour `1` is for the initial values.
Comments directly in the code.
Some observations:
- Most code is only called at the beginning of a day
- Structure of daily part: Day length, soil properties, soil biogeochemistry, vegetation high, vegetation low
- Vegetation high and low seem to share the same logic (duplicate code)
- The only part that is executed hourly is optional (and I'm not sure what it does), and optional part on wetlands

In [ ]:
for i=2:NN
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %! Print current iteration every 1000 steps
    if  (mod(i,1000) == 0) || (i == 2)
        disp('Iter:'); disp(i);
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %! indices prev. 24 hours
    pdind = [max(1,i-24):i-1]; %% previous day indexes 
    %%%
    %! new day begins (hour 0)
    if (Datam(i,4)==1) 
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        j=j+1; [jDay(j)]= JULIAN_DAY(Datam(i,:));
        %! Get daylight length
        [h_S,delta_S,zeta_S,T_sunrise,T_sunset,L_day(j)]= SetSunVariables(Datam(i,:),DeltaGMT,Lon,Lat,t_bef,t_aft); 
        clear h_S delta_S zeta_S T_sunrise T_sunset
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%% Biogeochemistry
        [Se_bio,Se_fc,Psi_bio,Tdp_bio,VSUM,VTSUM]=Biogeo_environment(Tdp(pdind,:),O(pdind,:),V(pdind,:),Soil_Param,Phy,SPAR,Bio_Zs);% sum(V(i,:))
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

        if OPT_SoilBiogeochemistry == 1
            IS= Ccrown*squeeze(ISOIL_L(j-1,:,:)) + Ccrown*squeeze(ISOIL_H(j-1,:,:));
            REXMY= Ccrown*squeeze(Rexmy_L(j-1,:,:)) + Ccrown*squeeze(Rexmy_H(j-1,:,:));
            FireA = 1*((sum(ManIH==-5) + sum(ManIL==-5)) > 0);
            [P(j,:),LEAK_NH4(j),LEAK_NO3(j),LEAK_P(j),LEAK_K(j),LEAK_DOC(j),LEAK_DON(j),LEAK_DOP(j),...
                R_NH4(j),R_NO3(j),R_P(j),R_K(j),R_DOC(j),R_DON(j),R_DOP(j),...
                Nuptake_H(j,:),Puptake_H(j,:),Kuptake_H(j,:),Nuptake_L(j,:),Puptake_L(j,:),Kuptake_L(j,:),RexmyI(j,:),...
                R_litter(j),R_microbe(j),R_litter_sur(j),R_ew(j),VOL(j),N2flx(j),Min_N(j),Min_P(j),...
                R_bacteria(j),RmycAM(j),RmycEM(j),Prod_B(j),Prod_F(j),BfixN(j),NavlI(j,:),LitFirEmi(j,:)]=BIOGEO_UNIT(P(j-1,:),IS,Zbio,sum(Bio_Zs.*rsd),PHs,Tdp_bio,mean(Ta(pdind)),Psi_bio,Se_bio,Se_fc,VSUM,VTSUM,...
                Ccrown,Bio_Zs,RfH_Zs,RfL_Zs,sum(Lk(pdind)),sum(Rd(pdind)),sum(Rh(pdind)),sum(Pr(pdind)),sum(T_H(pdind,:),1),sum(T_L(pdind,:),1),B_H(j-1,:,3),B_L(j-1,:,3),LAI_H(j-1,:),LAI_L(j-1,:),...
                SupN_H(j-1,:),SupP_H(j-1,:),SupK_H(j-1,:),SupN_L(j-1,:),SupP_L(j-1,:),SupK_L(j-1,:),...
                REXMY,RexmyI(j-1,:),ExEM,NavlI(j-1,:),Pcla,Psan,...
                B_IO,jDay(j),FireA,0);
            %%%%%
            %%%%%
            BLit(j,:)= 0.002*sum(P(j,1:5))*Ccrown; %% %%[kg DM / m2]
            Bam =  P(j,20); %%[gC/m2]
            Bem =  P(j,21); %%[gC/m2]
        else
            %! Do not calculate nutrient related variables
            %%%%%
            BLit(j,:)= 0.0 ; %  0.100 ; %% %%[kg DM / m2]
            %%%
            Nuptake_H(j,:)= 0.0;
            Puptake_H(j,:)= 0.0;
            Kuptake_H(j,:)= 0.0; %% [gK/m^2 day]
            %%%%
            Nuptake_L(j,:)= 0.0;  %% [gN/m^2 day]
            Puptake_L(j,:)= 0.0;
            Kuptake_L(j,:)= 0.0;
            %%%%
            NavlI(j,:)=[0 0 0];
            %P(j,:)=0.0;
            %LEAK_NH4(j)= 0.0; LEAK_NO3(j)= 0.0; LEAK_P(j)= 0.0; LEAK_K(j) = 0.0; LEAK_DOC(j)= 0.0;
            %R_litter(j)= 0.0; R_microbe(j)= 0.0; R_ew(j)=0; R_litter_sur(j)= 0.0; VOL(j)= 0.0; N2flx(j)= 0.0; BfixN(j)=0; LitFirEmi(j,:)=0;
            %LEAK_DOP(j)= 0.0; LEAK_DON(j)= 0.0; Min_N(j)=0; Min_P(j) =0; R_bacteria(j)=0; RmycAM(j)=0; RmycEM(j)=0;
            %RexmyI(j,:)=0.0;
            Bam=NaN; Bem=NaN;
        end


        %%% projected area n-coordinate
        for cc=1:length(Ccrown)
            %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            %%%%%%%%%%%%%%%%%%%%%%%%%
            %! High vegetation
            if (ZR95_H(cc) > 0) || (ZRmax_H(cc) > 0)
                %! Vegetation dynamics
                [LAI_H(j,cc),B_H(j,cc,:),NPP_H(j,cc),ANPP_H(j,cc),Rg_H(j,cc),RA_H(j,cc),Rms_H(j,cc),Rmr_H(j,cc),Rmc_H(j,cc),PHE_S_H(j,cc),...
                    dflo_H(j,cc),AgeL_H(j,cc),e_rel_H(j,cc),e_relN_H(j,cc),LAIdead_H(j,cc),NBLeaf_H(j,cc),Sr_H(j,cc),Slf_H(j,cc),Sfr_H(j,cc),Swm_H(j,cc),Sll_H(j,cc),Rexmy_H(j,cc,:),Rrootl_H(j,cc),...
                    AgeDL_H(j,cc),Bfac_dayH(j,cc),Bfac_weekH(j,cc),NPPI_H(j,cc),TdpI_H(j,cc),NupI_H(j,cc,:),PARI_H(j,cc,:),NBLI_H(j,cc),RB_H(j,cc,:),FNC_H(j,cc),Nreserve_H(j,cc),Preserve_H(j,cc),Kreserve_H(j,cc),...
                    rNc_H(j,cc),rPc_H(j,cc),rKc_H(j,cc),ManIH(cc)]= VEGGIE_UNIT(B_H(j-1,cc,:),PHE_S_H(j-1,cc),dflo_H(j-1,cc),AgeL_H(j-1,cc),AgeDL_H(j-1,cc),...
                    Ta(pdind),Tdp_H(pdind,cc),PARB(pdind)+PARD(pdind),Psi_x_H(pdind,cc),Psi_l_H(pdind,cc),An_H(pdind,cc),Rdark_H(pdind,cc),NPP_H(j-1,cc),jDay(j),Datam(i,:),...
                    NPPI_H(j-1,cc),TdpI_H(j-1,cc),Bfac_weekH(j-1,cc),NupI_H(j-1,cc,:),NavlI(j,:),PARI_H(j-1,cc,:),NBLI_H(j-1,cc),NBLeaf_H(j-1,cc),...
                    Lat,VegH_Param_Dyn,cc,...
                    Nreserve_H(j-1,cc),Preserve_H(j-1,cc),Kreserve_H(j-1,cc),Nuptake_H(j,cc),Puptake_H(j,cc),Kuptake_H(j,cc),FNC_H(j-1,cc),Se_bio,Tdp_bio,...
                    ParEx_H(cc),ExEM,Bam,Bem,Mpar_H(cc),TBio_Ht(j-1,cc),OPT_EnvLimitGrowth,OPT_VCA,OPT_VD,OPT_SoilBiogeochemistry);
                %%%%%%%%%%%%%%%%%%
                %! Plan exports
                [TexC_H(j,cc),TexN_H(j,cc),TexP_H(j,cc),TexK_H(j,cc),TNIT_H(j,cc),TPHO_H(j,cc),TPOT_H(j,cc),NuLit_H(j,cc,:),Nreserve_H(j,cc),Preserve_H(j,cc),Kreserve_H(j,cc),...
                    SupN_H(j,cc),SupP_H(j,cc),SupK_H(j,cc),ISOIL_H(j,cc,:)]= Plant_Exports(B_H(j,cc,:),B_H(j-1,cc,:),NuLit_H(j-1,cc,:),...
                    Slf_H(j,cc),Sfr_H(j,cc),Swm_H(j,cc),Sll_H(j,cc),Sr_H(j,cc),Rexmy_H(j,cc,:),Stoich_H(cc),Mpar_H(cc),fab_H(cc),fbe_H(cc),RB_H(j,cc,:),...
                    Nreserve_H(j,cc),Preserve_H(j,cc),Kreserve_H(j,cc),rNc_H(j,cc),rPc_H(j,cc),rKc_H(j,cc),ManIH(cc),OPT_SoilBiogeochemistry);
                %%%%%%%%%%%%%%%%
                
                %%%%%%%%%%%%%%%%%%%%%% Change in height SAI and Ccrown
                %! Stem Area Index
                SAI_H(j,cc) =SAI_H(j-1,cc);
                %! Grass
                if aSE_H(cc) == 2
                    [hc_H(j,cc)] = GrassHeight(LAI_H(j,cc),LAIdead_H(j,cc));
                %! Crops (?)
                elseif aSE_H(cc) == 5
                    [hc_H(j,cc),SAI_H(j,cc),B_H(j,:,:),Ccrown,Nreserve_H(j-1:j,:),Preserve_H(j-1:j,:),Kreserve_H(j-1:j,:),AgrHarNut(j,:)] = CropHeightType(LAI_H(j,cc),LAIdead_H(j,cc),cc,B_H(j,:,:),...
                        Ccrown,Nreserve_H(j,:),Preserve_H(j,:),Kreserve_H(j,:),ManIH,Mpar_H,VegH_Param_Dyn,OPT_SoilBiogeochemistry);
                    %%%%
                else
                    hc_H(j,cc)= hc_H(j-1,cc); %%%[m]
                    if OPT_VCA == 1
                        %%%%%%
                        [Ccrown_t(j,cc),hc_H(j,cc),SAI_H(j,cc),BA_H(j,cc),Tden_H(j,cc),AgePl_H(j,cc),TBio_Ht(j,cc)]=Vegetation_Structural_Attributes(dtd,...
                            Ccrown_t(j-1,cc),B_H(j,cc,:),fab_H(cc),Tden_H(j-1,cc),AgePl_H(j-1,cc),OPT_ALLOME);
                        %%%%
                        Ccrown(cc) = CcrownFIX(cc)*Ccrown_t(j,cc);
                        B_H(j,cc,:)= B_H(j,cc,:)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Nreserve_H(j,cc)=Nreserve_H(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Preserve_H(j,cc)=Preserve_H(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Kreserve_H(j,cc)=Kreserve_H(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                    end
                end
                if OPT_DROOT == 1 
                    %%%%%%%%%
                    [ZR95_H(cc),RfH_Zs(cc,:)]=Root_Depth_Dynamics(CASE_ROOT,B_H(j,cc,:),B_H(j-1,cc,:),Rrootl_H(j,cc),Zs,ZR95_H(cc),ZR50_H(cc),ZRmax_H(cc),...
                        Bfac_dayH(j,cc),Psan,Tdp_H(pdind,cc),O(pdind,:),Soil_Param,a_root_H(cc));
                end
                ZR95_Ht(j,cc)=ZR95_H(cc); 
                
            else
                %! Absence of high vegetation - variables are mostly set to 0
                LAI_H(j,cc)=0;B_H(j,cc,:)=0;NPP_H(j,cc)=0;ANPP_H(j,cc)=0;Rg_H(j,cc)=0;RA_H(j,cc)=0;Rms_H(j,cc)=0;
                Rmr_H(j,cc)=0;PHE_S_H(j,cc)=1;dflo_H(j,cc)=0;AgeL_H(j,cc)=0;e_rel_H(j,cc)=0; e_relN_H(j,cc)=0;
                SAI_H(j,cc)=0; hc_H(j,cc)=0;
                LAIdead_H(j,cc) =LAIdead_H(j-1,cc);
                Sr_H(j,cc)=0;Slf_H(j,cc)=0;Sfr_H(j,cc)=0;Swm_H(j,cc)=0; Rrootl_H(j,cc)=0;
                Rexmy_H(j,cc,:)=0;AgeDL_H(j,cc)=0;FNC_H(j,cc)=1; Nreserve_H(j,cc)=0;Preserve_H(j,cc)=0;Kreserve_H(j,cc)=0;
                rNc_H(j,cc)=1;rPc_H(j,cc)=1;rKc_H(j,cc)=1;
                Bfac_dayH(j,cc)=0; Bfac_weekH(j,cc)=0; NPPI_H(j,cc)=0; TdpI_H(j,cc)=0; NupI_H(j,cc,:)=0; RB_H(j,cc,:)=0;
                TexC_H(j,cc)=0;TexN_H(j,cc)=0;TexP_H(j,cc)=0;TexK_H(j,cc)=0;TNIT_H(j,cc)=0;TPHO_H(j,cc)=0;TPOT_H(j,cc)=0;
                ISOIL_H(j,cc,:)=0; NuLit_H(j,cc,:)=0;
            
            end
            %%%%%%%%%%%%%%%%%
            %! Low vegetation
            if (ZR95_L(cc) > 0) || (ZRmax_L(cc) > 0)
                [LAI_L(j,cc),B_L(j,cc,:),NPP_L(j,cc),ANPP_L(j,cc),Rg_L(j,cc),RA_L(j,cc),Rms_L(j,cc),Rmr_L(j,cc),Rmc_L(j,cc),PHE_S_L(j,cc),...
                    dflo_L(j,cc),AgeL_L(j,cc),e_rel_L(j,cc),e_relN_L(j,cc),LAIdead_L(j,cc),NBLeaf_L(j,cc),Sr_L(j,cc),Slf_L(j,cc),Sfr_L(j,cc),Swm_L(j,cc),Sll_L(j,cc),Rexmy_L(j,cc,:),Rrootl_L(j,cc),...
                    AgeDL_L(j,cc),Bfac_dayL(j,cc),Bfac_weekL(j,cc),NPPI_L(j,cc),TdpI_L(j,cc),NupI_L(j,cc,:),PARI_L(j,cc,:),NBLI_L(j,cc),RB_L(j,cc,:),FNC_L(j,cc),Nreserve_L(j,cc),Preserve_L(j,cc),Kreserve_L(j,cc),...
                    rNc_L(j,cc),rPc_L(j,cc),rKc_L(j,cc),ManIL(cc)]= VEGGIE_UNIT(B_L(j-1,cc,:),PHE_S_L(j-1,cc),dflo_L(j-1,cc),AgeL_L(j-1,cc),AgeDL_L(j-1,cc),...
                    Ta(pdind),Tdp_L(pdind,cc),PARB(pdind)+PARD(pdind),Psi_x_L(pdind,cc),Psi_l_L(pdind,cc),An_L(pdind,cc),Rdark_L(pdind,cc),NPP_L(j-1,cc),jDay(j),Datam(i,:),...
                    NPPI_L(j-1,cc),TdpI_L(j-1,cc),Bfac_weekL(j-1,cc),NupI_L(j-1,cc,:),NavlI(j,:),PARI_L(j-1,cc,:),NBLI_L(j-1,cc),NBLeaf_L(j-1,cc),...
                    Lat,VegL_Param_Dyn,cc,...
                    Nreserve_L(j-1,cc),Preserve_L(j-1,cc),Kreserve_L(j-1,cc),Nuptake_L(j,cc),Puptake_L(j,cc),Kuptake_L(j,cc),FNC_L(j-1,cc),Se_bio,Tdp_bio,...
                    ParEx_L(cc),ExEM,Bam,Bem,Mpar_L(cc),TBio_Lt(j-1,cc),OPT_EnvLimitGrowth,OPT_VCA,OPT_VD,OPT_SoilBiogeochemistry);
                %%%%%%%%%%%%%%%%%%%
                [TexC_L(j,cc),TexN_L(j,cc),TexP_L(j,cc),TexK_L(j,cc),TNIT_L(j,cc),TPHO_L(j,cc),TPOT_L(j,cc),NuLit_L(j,cc,:),Nreserve_L(j,cc),Preserve_L(j,cc),Kreserve_L(j,cc),...
                    SupN_L(j,cc),SupP_L(j,cc),SupK_L(j,cc),ISOIL_L(j,cc,:)]= Plant_Exports(B_L(j,cc,:),B_L(j-1,cc,:),NuLit_L(j-1,cc,:),...
                    Slf_L(j,cc),Sfr_L(j,cc),Swm_L(j,cc),Sll_L(j,cc),Sr_L(j,cc),Rexmy_L(j,cc,:),Stoich_L(cc),Mpar_L(cc),fab_L(cc),fbe_L(cc),RB_L(j,cc,:),...
                    Nreserve_L(j,cc),Preserve_L(j,cc),Kreserve_L(j,cc),rNc_L(j,cc),rPc_L(j,cc),rKc_L(j,cc),ManIL(cc),OPT_SoilBiogeochemistry);
                
                %%%%%%%%%%%%%%%%%%%%%% Change in height SAI and Ccrown
                SAI_L(j,cc) =SAI_L(j-1,cc);
                if aSE_L(cc) == 2
                    [hc_L(j,cc)] = GrassHeight(LAI_L(j,cc),LAIdead_L(j,cc));
                elseif aSE_L(cc) == 5
                    [hc_L(j,cc),SAI_L(j,cc),B_L(j,:,:),Ccrown,Nreserve_L(j,:),Preserve_L(j,:),Kreserve_L(j,:),AgrHarNut(j,:)] = CropHeightType(LAI_L(j,cc),LAIdead_L(j,cc),cc,B_L(j,:,:),...
                        Ccrown,Nreserve_L(j,:),Preserve_L(j,:),Kreserve_L(j,:),ManIL,Mpar_L,VegL_Param_Dyn,OPT_SoilBiogeochemistry);
                else
                    hc_L(j,cc)= hc_L(j-1,cc); %%%[m]
                    if OPT_VCA == 1
                        %%%%%%
                        [Ccrown_t(j,cc),hc_L(j,cc),SAI_L(j,cc),BA_L(j,cc),Tden_L(j,cc),AgePl_L(j,cc),TBio_Lt(j,cc)]=Vegetation_Structural_Attributes(dtd,...
                            Ccrown_t(j-1,cc),B_L(j,cc,:),fab_L(cc),Tden_L(j-1,cc),AgePl_L(j-1,cc),OPT_ALLOME);
                        %%%%
                        Ccrown(cc) = CcrownFIX(cc)*Ccrown_t(j,cc);
                        B_L(j,cc,:)= B_L(j,cc,:)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Nreserve_L(j,cc)=Nreserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Preserve_L(j,cc)=Preserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                        Kreserve_L(j,cc)=Kreserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
                    end
                end

                if OPT_DROOT == 1 
                    %%%%%%%%%
                    [ZR95_L(cc),RfL_Zs(cc,:)]=Root_Depth_Dynamics(CASE_ROOT,B_L(j,cc,:),B_L(j-1,cc,:),Rrootl_L(j,cc),Zs,ZR95_L(cc),ZR50_L(cc),ZRmax_L(cc),...
                        Bfac_dayL(j,cc),Psan,Tdp_L(pdind,cc),O(pdind,:),Soil_Param,a_root_L(cc));
                end
                ZR95_Lt(j,cc)=ZR95_L(cc); 

            else
                LAI_L(j,cc)=0;B_L(j,cc,:)=0;NPP_L(j,cc)=0;ANPP_L(j,cc)=0;Rg_L(j,cc)=0;RA_L(j,cc)=0;Rms_L(j,cc)=0;
                Rmr_L(j,cc)=0;PHE_S_L(j,cc)=1;dflo_L(j,cc)=0;AgeL_L(j,cc)=0;e_rel_L(j,cc)=0; e_relN_L(j,cc)=0;
                SAI_L(j,cc)=0; hc_L(j,cc)=0;
                LAIdead_L(j,cc) =LAIdead_L(j-1,cc);
                Sr_L(j,cc)=0;Slf_L(j,cc)=0;Sfr_L(j,cc)=0;Swm_L(j,cc)=0;  Rrootl_L(j,cc)=0;
                Rexmy_L(j,cc,:)=0;AgeDL_L(j,cc)=0;FNC_L(j,cc)=1;Nreserve_L(j,cc)=0;Preserve_L(j,cc)=0;Kreserve_L(j,cc)=0;
                rNc_L(j,cc)=1;rPc_L(j,cc)=1;rKc_L(j,cc)=1;
                Bfac_dayL(j,cc)=0; Bfac_weekL(j,cc)=0; NPPI_L(j,cc)=0; TdpI_L(j,cc)=0;  NupI_L(j,cc,:)=0; RB_L(j,cc,:)=0;
                TexC_L(j,cc)=0;TexN_L(j,cc)=0;TexP_L(j,cc)=0;TexK_L(j,cc)=0;TNIT_L(j,cc)=0;TPHO_L(j,cc)=0;TPOT_L(j,cc)=0;
                ISOIL_L(j,cc,:)=0; NuLit_L(j,cc,:)=0;
            end
            %%%%%%%%%%%%%%%%%
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        if OPT_VCA == 0  %%% Solutions for Crops aSE==5 
            Ccrown_t(j,:) = Ccrown;
            Cbare = 1 - sum(Ccrown) - Cwat - Curb - Crock;
            Check_Land_Cover_Fractions(Crock,Curb,Cwat,Cbare,Ccrown);
        end
    end
    %%%%%%% %%%%%%%

    %! Only this part is executed hourly
    %%%%%%% %%%%%%%
    %! Not sure what this is about
    if OPT_VCA == 1
        if((OPT_ALLOME == 1)&& (ZR95_L(2))>0) %%% Ad hoc solution for oil palm
            Ccrown_t(j,2) = 1-Ccrown_t(j,1);  % understory of Oil Palm
            Ccrown(2) = Ccrown_t(j,2);
            %%% Theoretically to preserve mass // but this is considered
            %%% lost understory in the  oil palm 
            %B_L(j,cc,:)= B_L(j,cc,:)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
            %Nreserve_L(j,cc)=Nreserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
            %Preserve_L(j,cc)=Preserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
            %Kreserve_L(j,cc)=Kreserve_L(j,cc)*Ccrown_t(j-1,cc)/Ccrown_t(j,cc);
        end
        Cbare = 1 - sum(Ccrown);
        if zatm < (max(max(hc_H),max(hc_L))+2)
            zatm = zatm + 2;
        end
    end

    Qi_in(i,:)=Qi_out(i-1,:);
    q_runon(i)=0; %Rd(i-1)+Rh(i-1);
    if OPT_WET == 1 %%% Wetland option with water standing in the surface
        q_runon(i)=Wlev(i);
        if isnan(Wlev(i))
            q_runon(i)=Wlevm1;
        end
        NetWatWet(i) = q_runon(i) - Wlevm1 ;
    end

#### Lines 558-659: Wrap Up
Print computational time and perform checks to ensure preservation of mass (?), energy, carbon and nutrients.

### Finalize simulation
To finalize the simulation, the prova file will export the variables from the workspace to a file on disk, an navigate back to the original directory:

In [ ]:
file_to_save = strcat('../results/Ris_',id_location,'.mat'); %! <- set output file here
save(file_to_save);
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%
cd(current_directory);

## Description of other files

### Part of simulation
##### JULIAN_DAY
Count the day of the year

##### SetSunVariables
Compute sun position (declination, azimuth, altitude), sunrise and -set, day length
! Duplicates code of `JULIAN_DAY`

#### Biogeochemistry related
##### Biogeo_environment
Soil biogechemistry layer related variables: Moisture, volume, temperature. Seems to be run also when biogechemistry module is off.

##### BIOGEO_UNIT
Calculates a (large) number of biogeochem. related variables. Only if OPT_SoilBiogeochemistry is 1.
Also calls `Biogeo_Leakage`, `BIOGEOCHEMISTRY_DYNAMIC3`, `Biogeochemistry_parameter`

##### Biogeo_Leakage
Leakage of nutrients from soil

##### BIOGEOCHEMISTRY_DYNAMIC3
Nutrient cycles...?

##### Biogeochemistry_parameter
Instantiate a struct from hard-coded values

#### Vegetation dynamics
##### VEGGIE_UNIT
Vegetation dynamics?
Calls `BetaFactor`, `EnvConGrowth`, `Nutrients_Available`, `VEG_DYN_RES`, `RELATIVE_PC`, `Grass_Cut_Grazing`, `Crop_Planting_Harvest`, `Forest_Logging_Fire`, `Root_properties`

##### Plant_Exports

##### GrassHeight
Estimate grass height

##### CropHeightType
Calculate a range of crop-related vegetation characteristics (height, SAI, mobile nutrient reserves,...)

##### Root_Depth_Dynamics
Only run if `OPT_DROOT` is 1

#### Sanity checks
##### Check_Land_Cover_Fractions
Check whether land cover fractions add up to 1